# İş Bankası Machine Learning challenge #5

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder


In [ ]:
Train = pd.read_parquet('/kaggle/input/turkiye-is-bankasi-ml-challenge-5/train_final.parquet')
Test = pd.read_parquet('/kaggle/input/turkiye-is-bankasi-ml-challenge-5/test_final.parquet')
Final = pd.read_parquet('/kaggle/input/turkiye-is-bankasi-ml-challenge-5/submission_sample_final.parquet')

In [ ]:
Train.head()

In [ ]:
Test.insert(loc=4, column='n_seconds', value=0)
Train.insert(loc=4, column='n_seconds', value=0)
Train['n_seconds'] = Train['n_seconds_1'] + Train['n_seconds_2'] + Train['n_seconds_3']
Test['n_seconds'] = Test['n_seconds_1'] + Test['n_seconds_2'] + Test['n_seconds_3']
Train=Train.drop([ 'carrier', 'devicebrand','n_seconds_1','n_seconds_2','n_seconds_3'], axis=1)
Test=Test.drop([ 'carrier', 'devicebrand','n_seconds_1','n_seconds_2','n_seconds_3'], axis=1)
Train.head()

In [ ]:
Train['target'] = Train['target'].apply(lambda x: ''.join(['1' if f'menu{i}' in x else '0' for i in range(1, 10)]))

In [ ]:
Train.head()

In [ ]:
#hedef ve ozellik sutunlarini ayiriyoruz
X = Train.iloc[:, 1:-1]  
y = Train.iloc[:, -1:] 
X.head()

In [ ]:
y.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
y_train = np.array(y_train).ravel()
y_test = np.array(y_test).ravel()

In [ ]:

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)


In [ ]:
y_train

In [ ]:
model = xgb.XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
y_pred = label_encoder.inverse_transform(y_pred)

accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy : {accuracy}")

In [ ]:
from sklearn.metrics import jaccard_score

In [ ]:
jaccard_score(y_test, y_pred)

In [ ]:
Test_Test= Test.iloc[:, 1:]  

In [ ]:
Test_Test.head()

In [ ]:
test_model=model.predict(Test_Test)

In [ ]:
test_model = label_encoder.inverse_transform(test_model)


In [ ]:
test_model

In [ ]:
results_df = pd.DataFrame({"id":'','target': test_model })

results_df.head()

In [ ]:
results_df['id'] = Test['id']

In [ ]:
results_df.head()

In [ ]:
results_df.dtypes

In [ ]:
for index, row in results_df.iterrows():
    target = row["target"]
    total = sum(map(int, list(target)))  

    if total != 3:
        print("id:", row["id"], " target:", row["target"])

In [ ]:
results_df.to_csv('submission.csv', index=False)